## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_DataBase/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,82.45,78,20,12.66,few clouds
1,1,Filadelfia,US,39.9523,-75.1638,53.49,23,0,13.80,clear sky
2,2,Ushuaia,AR,-54.8000,-68.3000,35.26,87,20,4.61,few clouds
3,3,Victoria,HK,22.2855,114.1577,81.97,85,100,18.01,overcast clouds
4,4,Mayo,US,38.8876,-76.5119,52.11,66,0,1.01,clear sky


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Sao Filipe,CV,14.8961,-24.4956,70.95,69,9,8.03,clear sky
10,10,Tucuma,BR,-0.1833,-50.0500,75.78,95,100,4.70,overcast clouds
14,14,Bambous Virieux,MU,-20.3428,57.7575,76.21,83,75,2.30,broken clouds
17,17,Puerto Ayora,EC,-0.7393,-90.3518,71.38,92,55,5.19,broken clouds
27,27,Rikitea,PF,-23.1203,-134.9692,77.13,68,100,16.15,overcast clouds
29,29,Arraial Do Cabo,BR,-22.9661,-42.0278,75.11,87,3,9.98,clear sky
30,30,Vao,NC,-22.6667,167.4833,74.01,76,100,24.92,overcast clouds
38,38,Vaini,TO,-21.2000,-175.2000,73.56,100,100,5.75,light rain
41,41,Ciudad Guayana,VE,8.3533,-62.6528,78.53,91,44,5.53,scattered clouds
48,48,Estelle,US,29.8458,-90.1067,75.20,68,0,4.61,clear sky


In [24]:
# 4a. Determine if there are any empty rows.
# preferred_cities_df.isnull().sum()
# preferred_cities_df.notnull().sum()
# preferred_cities_df.count()
city_data_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [75]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = pd.DataFrame(data=city_data_df)
clean_df=clean_df.dropna()

clean_preferred_cities_df = clean_df.loc[(clean_df["Max Temp"] <= max_temp) & (clean_df["Max Temp"] >= min_temp)]
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [84]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Sao Filipe,CV,70.95,clear sky,14.8961,-24.4956,
10,Tucuma,BR,75.78,overcast clouds,-0.1833,-50.0500,
14,Bambous Virieux,MU,76.21,broken clouds,-20.3428,57.7575,
17,Puerto Ayora,EC,71.38,broken clouds,-0.7393,-90.3518,
27,Rikitea,PF,77.13,overcast clouds,-23.1203,-134.9692,
29,Arraial Do Cabo,BR,75.11,clear sky,-22.9661,-42.0278,
30,Vao,NC,74.01,overcast clouds,-22.6667,167.4833,
38,Vaini,TO,73.56,light rain,-21.2000,-175.2000,
41,Ciudad Guayana,VE,78.53,scattered clouds,8.3533,-62.6528,
48,Estelle,US,75.20,clear sky,29.8458,-90.1067,


In [85]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [86]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!="")]
clean_hotel_df.count()

City                   124
Country                124
Max Temp               124
Current Description    124
Lat                    124
Lng                    124
Hotel Name             124
dtype: int64

In [87]:
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Sao Filipe,CV,70.95,clear sky,14.8961,-24.4956,Hotel Xaguate
14,Bambous Virieux,MU,76.21,broken clouds,-20.3428,57.7575,Casa Tia Villa
17,Puerto Ayora,EC,71.38,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
27,Rikitea,PF,77.13,overcast clouds,-23.1203,-134.9692,People ThankYou
29,Arraial Do Cabo,BR,75.11,clear sky,-22.9661,-42.0278,Pousada Porto Praia
30,Vao,NC,74.01,overcast clouds,-22.6667,167.4833,Hotel Kou-Bugny
38,Vaini,TO,73.56,light rain,-21.2000,-175.2000,Keleti Beach Resort
41,Ciudad Guayana,VE,78.53,scattered clouds,8.3533,-62.6528,Hotel Las Americas
48,Estelle,US,75.20,clear sky,29.8458,-90.1067,Boomtown Casino & Hotel New Orleans
50,Beyneu,KZ,74.61,overcast clouds,45.3167,55.2000,NUR


In [88]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [89]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [90]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))